In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import random
import matplotlib.pyplot as plt

In [2]:
data_clean = pd.read_pickle("data_clean_allrecipe.pkl")
data_clean.head()

,ingredient
ID,
BBC1,unsalted_butter chocolate plain_flour cocoa_po...
BBC10,butter_biscuit butter syrup dark_chocolate mil...
BBC100,digestive_biscuit butter caramel salt chocolat...
BBC1000,unsalted_butter hot_water chocolate_cocoa cast...
BBC1001,plain_flour ginger unsalted_butter muscovado_s...


In [3]:
# list of text documents
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(data_clean['ingredient'])
# summarize
#print(vectorizer.vocabulary_)
#print(vectorizer.idf_)
# encode document
vector = vectorizer.transform(data_clean['ingredient']) # this is sparse vector
#print(vector)
# summarize encoded vector
#print(vector.shape)
#print(vector.toarray())
terms = vectorizer.get_feature_names()
terms
sums = vector.sum(axis=0)
# connecting term to its sums frequency
data = []
for col, term in enumerate(terms):
    data.append( (term, sums[0,col] ))

ranking = pd.DataFrame(data, columns=['term','rank'])
print(ranking.sort_values('rank', ascending=False))

                          term         rank
3513                     sugar  2023.351620
1224                       egg  1545.392928
382                     butter  1531.718240
1402                     flour  1513.851328
3239                      salt  1389.773572
3887                   vanilla  1211.948133
2329                      milk   988.125926
742                   cinnamon   908.944744
157              baking_powder   860.261038
3909           vanilla_extract   788.642152
4003                     water   768.982005
366                brown_sugar   741.241354
158                baking_soda   710.983843
2281                 margarine   562.732018
2082               lemon_juice   561.102624
957               cream_cheese   547.486785
1243                 egg_white   528.721888
1247                  egg_yolk   518.233576
1725          granulated_sugar   505.763098
2524                    nutmeg   490.627666
2708                     pecan   489.961484
2850            powdered_sugar  

In [4]:
#Since most cells in this matrix will be zero, this will give the percentage of cells containing non-zero values.
# Materialize the sparse data
data_dense = vector.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.20033588914359896 %


In [5]:
#Lets buil LDA model
from sklearn.decomposition import LatentDirichletAllocation

no_topics = 100

# Run LDA
lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=10, learning_method='online', learning_decay=0.7,learning_offset=50.,random_state=100).fit(vector)
lda_output = lda_model.fit(vector)
lda_output

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=100, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [6]:
#This is to diagnose model performance with perplexity and log-likelihood
#Note: The model with higher log-likelihood and lower perplexity ( (exp(-1. * log-likelihood per word)) is considered to be good).
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(vector))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(vector))

# See model parameters
#pprint(lda_model.get_params())

Log Likelihood:  -644007.7829965665
Perplexity:  73529.6426427001


In [7]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ,".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [8]:
no_top_words = 10
display_topics(lda_model, terms, no_top_words)

Topic 0:
base ,center ,apricot_sauce ,dark_chocolate_fudge ,paste_food_coloring ,nut_cooky ,dry_cashew ,carrot_pineapple_cooky ,apple_oatmeal_cooky ,apricot_cashew_cooky
Topic 1:
pistachio_nut ,biscuit ,orange_water ,milk_cheese ,red_plum ,pate_dough ,white_corn_meal ,vanilla_pastry ,pulp ,black_food
Topic 2:
confect_sugar ,caramel_wafer ,toffee ,peanut ,vanilla_ice_cream ,banana ,milk ,soft_sugar ,white_marzipan ,nutmeg_heart
Topic 3:
jellied_cranberry_sauce ,coconut_cooky ,soda_filling ,raspberry ,blueberry ,easy_vanilla ,strawberry ,liqueur ,caster_sugar ,icing_sugar
Topic 4:
swiss_milk_pudding ,red_cinnamon ,unsalted_margarine ,banana_liqueur ,chocolate_peanut ,basil ,dark_cocoa ,butternut_flavoring ,cinnamon_graham_cracker ,fruit_apricot_jam
Topic 5:
chocolate_chip ,cheese ,chocolate ,butterscotch_chip ,ladyfinger ,peanut_butter_chip ,cocoa_powder ,matzo_meal ,chow_noodle ,candied_fruit
Topic 6:
unflavored_gelatin ,fruit_juice ,fat_free_cheese ,cordial ,sherbet ,unsweetened_orange

In [9]:
#to visualize the LDA model with pyLDAvis
#Note: A good topic model will have non-overlapping, fairly big sized blobs for each topic. 
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline



pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, vector, vectorizer, mds='tsne')
panel

C:\Users\ReetuSingh\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
39     -33.514404  -86.950455       1        1  54.974522
55     -27.824888  -89.891228       2        1   4.835300
88     -41.266277  -82.664932       3        1   3.066217
43     -19.561357   69.330795       4        1   2.139997
76      20.050680   82.068596       5        1   1.389239
5      -75.284241   28.848154       6        1   1.217159
7       79.971718   30.027391       7        1   0.999517
95      29.827845   57.013165       8        1   0.876947
77      -1.508568   77.436180       9        1   0.831631
61      -9.236195   27.755823      10        1   0.692267
65      22.290537  -19.779293      11        1   0.568949
90      24.926193   27.777477      12        1   0.564642
38     -75.405891    2.460468      13        1   0.563246
35     363.464874 -134.196564      14        1   0.554169
6       57.530792  388.203918      15        1   0.545094
31     -29.000626   10.462585      16        1   0.524755
85     -24.544888   22.155058      17        1   0.475565
83     -29.257011  -12.462504      18        1   0.473427
96    -310.769501  231.153320      19        1   0.439851
89     147.057388 -351.086365      20        1   0.436676
1      -56.379944   13.177781      21        1   0.401116
24      32.734367  -51.175797      22        1   0.396878
48     -35.473854  -31.065323      23        1   0.393166
93      81.704514   -3.865488      24        1   0.390479
26      52.380398    7.139563      25        1   0.352567
42      45.742165   48.618149      26        1   0.348555
52      69.767761  -30.451624      27        1   0.340012
23     -71.148003  -20.933689      28        1   0.337902
80     -57.826191   52.004990      29        1   0.322838
53     -54.024170  -30.749153      30        1   0.313936
...           ...         ...     ...      ...        ...
8      -19.727085   33.779095      71        1   0.286278
28      29.864388   -9.283213      72        1   0.285871
49       2.393054   29.493332      73        1   0.285835
27     -16.268528  -10.287320      74        1   0.285737
92      13.348017   30.458914      75        1   0.285722
94      -8.118181  -17.360821      76        1   0.285684
34     -25.133505   -0.804014      77        1   0.285676
82      -5.579732   38.399258      78        1   0.285598
99       2.637858  -23.149757      79        1   0.285563
40     -17.068373    9.194968      80        1   0.285488
67     -13.012450   17.917315      81        1   0.285174
47      17.444626   18.905802      82        1   0.285054
57      11.173259  -15.660584      83        1   0.285042
60      26.603392    1.205243      84        1   0.285024
2      -14.676348   -0.285018      85        1   0.284963
79      -2.344310   19.338837      86        1   0.284962
44      16.983280    1.318829      87        1   0.284930
86      21.818773   10.202107      88        1   0.284907
17       7.822042   20.014933      89        1   0.284905
22      18.515575   -8.185802      90        1   0.284863
19       1.514557  -12.414165      91        1   0.284852
36      -6.105482   -6.502150      92        1   0.284122
46      -5.735144   10.546472      93        1   0.284122
78       9.335307   -5.799082      94        1   0.284122
74       1.540896   -3.128752      95        1   0.284122
71       3.721033   12.074423      96        1   0.284122
11       1.093276    4.898882      97        1   0.284122
14      -6.398765    2.205311      98        1   0.284122
54      11.928459   10.089781      99        1   0.284122
10       8.572242    2.605989     100        1   0.284122

[100 rows x 5 columns], topic_info=      Category         Freq                    Term        Total  loglift  \
term                                                                        
3811   Default   226.000000      unflavored_gelatin   226.000000  30.0000   
3513   Default  1525.000000                   su

In [10]:
# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_topics)]

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = terms
df_topic_keywords.index = topicnames

# View
df_topic_keywords

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
#To see the dominant topic in each document
#Note: To classify a document as belonging to a particular topic, a logical approach is to see which topic has the highest contribution to that document and assign it.

# index names
docnames = ["Doc" + str(i) for i in range(len(data_clean))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [ ]:
#Time to test the model
text = input("write the ingredients to learn about the recipe (using comma seperator)")
data_test = (text)
print("You have typed input '%s':" %data_test)

In [ ]:
d = np.argmax(lda_model.transform(vectorizer.transform([data_test],)))
print(d)

In [ ]:
no_top_words = 8
for topic_idx, topic in enumerate(lda_model.components_):
    if topic_idx== d:
        print ("Topic %d:" % (d))
        print (" ,".join([terms[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    pass